# Data Project Work
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd

## Data File Exploration

In [2]:
FILENAME = 'Energy Indicators.xls'
energy = pd.read_excel(
    # Filename
    FILENAME,
    # Name of columns (there are 4)
    names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'],
    # No headers
    header=None,
    # Load 4 columns excluding first two
    usecols=range(2,6),
    # Exlude the first 17th rows
    skiprows=18,
    # Null values
    na_values=['...'],
    # Number of rows to parse
    nrows=227
)

energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321.0,10.0,78.669280
1,Albania,102.0,35.0,100.000000
2,Algeria,1959.0,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.0,121.0,88.695650


In [3]:
energy.tail()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
222,Viet Nam,2554.0,28.0,45.32152
223,Wallis and Futuna Islands,0.0,26.0,0.00000
224,Yemen,344.0,13.0,0.00000
225,Zambia,400.0,26.0,99.71467
226,Zimbabwe,480.0,32.0,52.53612


In [4]:
# Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule)
energy['Energy Supply'] = energy['Energy Supply'] * 1000000

energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650


Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.
```

In [5]:
import re
from string import digits

remove_digits = str.maketrans('', '', digits)

In [6]:
test_tring = 'Clean (all in parentheses) test 24'

def clean_country_name(name: str) -> str:
    regex = re.compile(".*?\((.*?)\)")
    result = re.findall(regex, name)

    name = name.translate(remove_digits)

    for match in result:
        name = name.replace(
          match, ''
        ).replace(
          '()', ''
        ).replace(
          '  ', ' '
        )
    
    if name == 'United States of America':
        return 'United States'
    elif name == 'United Kingdom of Great Britain and Northern Ireland':
        return 'United Kingdom'
    elif name == 'China, Hong Kong Special Administrative Region':
        return 'Hong Kong'
      
    return str.strip(name)

result = clean_country_name(test_tring)

result

'Clean test'

In [7]:
energy['Country'] = energy['Country'].transform(clean_country_name)

## Checking

In [8]:
energy['Country'][
  (energy['Country'] == 'United States of America') |
  (energy['Country'] == 'United Kingdom of Great Britain and Northern Ireland') |
  (energy['Country'] == 'China, Hong Kong Special Administrative Region')
]

Series([], Name: Country, dtype: object)

In [9]:
energy['Country'][
  (energy['Country'] == 'United States') |
  (energy['Country'] == 'United Kingdom') |
  (energy['Country'] == 'Hong Kong') |
  (energy['Country'] == 'Bolivia') |
  (energy['Country'] == 'Switzerland')
]

24            Bolivia
43          Hong Kong
197       Switzerland
214    United Kingdom
216     United States
Name: Country, dtype: object

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

In [10]:
GDP = pd.read_csv('world_bank.csv', skiprows=4)

GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


In [11]:
def change_gdp_name(name):
    change = {
        "Korea, Rep.": "South Korea",  "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong",
    }
    
    if name in change:
        return change[name]
    
    return name

GDP['Country Name'] = GDP['Country Name'].transform(change_gdp_name)

## Checking

In [12]:
GDP['Country Name'][
    (GDP['Country Name'] == "South Korea") |
    (GDP['Country Name'] == "Iran") |
    (GDP['Country Name'] == "Hong Kong") 
]

93       Hong Kong
109           Iran
123    South Korea
Name: Country Name, dtype: object

Finally, load the Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology from the file scimagojr-3.xlsx, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame ScimEn.

In [13]:
ScimEn = pd.read_excel('scimagojr-3.xlsx', index_col='Rank')

ScimEn.head()

,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Rank,,,,,,,
1,China,127050,126767,597237,411683,4.70,138
2,United States,96661,94747,792274,265436,8.20,230
3,Japan,30504,30287,223024,61554,7.31,134
4,United Kingdom,20944,20357,206091,37874,9.84,139
5,Russian Federation,18534,18301,34266,12422,1.85,57


Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

This function should return a DataFrame with 20 columns and 15 entries.

In [14]:
GDP = GDP.rename(columns={'Country Name': 'Country'})

data = pd.merge(ScimEn.iloc[:15].reset_index(), energy, on='Country', how='left')

data = pd.merge(
    data, 
    GDP[['Country', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']], 
    on='Country'
)

data = data.set_index('Country')

data

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [15]:
data.shape

(15, 20)

In [16]:
def answer_one():
    return data

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [17]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [18]:
ScimEn.shape

(191, 7)

In [19]:
energy.shape

(227, 4)

In [20]:
GDP.shape

(264, 60)

In [21]:
def answer_two():
    return GDP.shape[0] - 15

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [22]:
Top15 = answer_one()

gdpColumns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

Top15[gdpColumns]

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,
China,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [23]:
from collections import defaultdict

avgGDPData = defaultdict(int)

for country in Top15.index:
    avgGDPData[country] = Top15.loc[country][gdpColumns].mean()
    
avgGDP = pd.Series(avgGDPData, index=Top15.index).sort_values(ascending=True)

avgGDP

Country
Iran                  4.441558e+11
South Korea           1.106715e+12
Australia             1.164043e+12
Spain                 1.418078e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
India                 1.769297e+12
Italy                 2.120175e+12
Brazil                2.189794e+12
United Kingdom        2.487907e+12
France                2.681725e+12
Germany               3.493025e+12
Japan                 5.542208e+12
China                 6.348609e+12
United States         1.536434e+13
dtype: float64

In [24]:
def answer_three():
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [25]:
avgGDP.index[5]

'Canada'

In [26]:
Top15.iloc[5][gdpColumns].var()

7.011532567290382e+21

In [27]:
def answer_four():
    return Top15.iloc[5][gdpColumns].var()

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [28]:
Top15['Energy Supply per Capita'].mean()

153.07142857142858

In [29]:
def answer_five():
    return Top15['Energy Supply per Capita'].mean()

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [30]:
result = Top15['% Renewable'].sort_values(ascending=False)

result

Country
Brazil                69.648030
Canada                61.945430
Spain                 37.968590
Italy                 33.667230
China                 19.754910
Germany               17.901530
Russian Federation    17.288680
France                17.020280
India                 14.969080
Australia             11.810810
United States         11.570980
United Kingdom        10.600470
Japan                 10.232820
Iran                   5.707721
South Korea                 NaN
Name: % Renewable, dtype: float64

In [31]:
result[0]

69.64803

In [32]:
result.index[0]

'Brazil'

In [33]:
def answer_six():
    return (
        result.index[0], 
        result[0]
    )

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [34]:
radio = (Top15['Self-citations']/Top15['Citations']).sort_values(ascending=False)

radio

Country
China                 0.689313
Russian Federation    0.362517
United States         0.335031
Iran                  0.332782
India                 0.288973
Japan                 0.275997
Italy                 0.238364
Brazil                0.237159
France                0.218943
South Korea           0.197035
Germany               0.195111
Spain                 0.194299
Canada                0.190369
United Kingdom        0.183773
Australia             0.171939
dtype: float64

In [35]:
radio.index[0]

'China'

In [36]:
def answer_seven():
    return(radio.index[0], radio[0])

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [37]:
result = (Top15['Energy Supply']/Top15['Energy Supply per Capita']).sort_values(ascending=False)

result

Country
China                 1.367645e+09
India                 1.276731e+09
United States         3.176154e+08
Brazil                2.059153e+08
Russian Federation    1.435000e+08
Japan                 1.274094e+08
Germany               8.036970e+07
Iran                  7.707563e+07
United Kingdom        6.387097e+07
France                6.383735e+07
Italy                 5.990826e+07
Spain                 4.644340e+07
Canada                3.523986e+07
Australia             2.331602e+07
South Korea                    NaN
dtype: float64

In [38]:
print(result.index[0])
print(result[0])

China
1367645161.2903225


In [39]:
def answer_eight():
    return (result.index[0], result[0])